<a href="https://colab.research.google.com/github/julialromero/regional-representations-graph-model/blob/master/dataset/earth_engine/download_landsat_imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip install pydrive

In [1]:
# ! pip install geemap

In [45]:
import sys
import os


import geemap
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

In [46]:
import ee
# ee.Authenticate()
ee.Initialize()

In [47]:
def makeRectangle(xy, xDistance, yDistance, proj, geoid):
    x = xy[0]
    y = xy[1]
    geometry = ee.Geometry.Rectangle(
                  [x - xDistance, 
                 
                   y - yDistance,
                   x + xDistance, 
                   y + yDistance], proj, False)
    
    return ee.Feature(geometry).set({'GEOID': geoid})


xDistance = 2434
yDistance = 2434
threshold_size = ee.Number(xDistance*yDistance)


In [48]:
def get_img(geoid):
    feat = tracts.filter(ee.Filter.eq('GEOID', geoid)).first()
    roi = feat.geometry()
    
    collection = (
      ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")  # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1
      .filterBounds(roi)
      .filter(ee.Filter.contains('.geo', roi))
      .filter(ee.Filter.calendarRange(6, 10, 'month'))
      .filterDate('2016-01-01', '2023-01-01')
      .sort('CLOUD_COVER')
    )

    top = collection.limit(1)

    # take the median pixel value; clip to the census geometry 
    # composite = top.median().clip(roi)
#     print(roi)
    img = top.first().clip(roi)#.reproject(crs='EPSG:4326')
    return img#, geoid
    

def createDir(dirPath):
  try:
    os.makedirs(dirPath)
  except:
    pass

def init_dir(subdir, fn):
  if os.path.exists(subdir):
      if fn in os.listdir(subdir):
#         print('pass')
        return True
     
  if not os.path.exists(subdir):
      print('creating directory')
      createDir(subdir)
      return True

In [49]:
tract_data_dir = r'../Tracts/nyc_metro_boundaries/nyc_metro_boundaries.shp'
temp_tract_data_dir = "../Tracts/nyc_metro_boundaries/temp.shp"
temp_envelope_data_dir = "../Tracts/nyc_metro_boundaries/temp_envelope.shp"

df = gpd.read_file(tract_data_dir)
df.rename(columns={'GEOID_TRAC': 'GEOID'}, inplace=True)  
# print(df.head())
shapefile = df[['GEOID', 'geometry']] 
# shapefile['geometry'] = shapefile['geometry'].envelope

In [50]:
earth_engine_filename = 'sentinel2.tif'
fn = earth_engine_filename

In [51]:
# clip or not clip the images to census polygon
earth_engine_clip_image_to_geometry = False

# Run to download Landsat Images

In [9]:
# https://stackoverflow.com/questions/60748584/create-ee-geometry-rectangle-centered-at-coordinate-point-and-specify-size-in-pi

In [14]:
save_dir = 'download_landsat_images'
count = 0
interval_size = 20
for interval in range(0, shapefile.shape[0], interval_size): 
    try:
        sub = shapefile.iloc[interval:interval+interval_size]
    except:
        sub = shapefile.iloc[interval:-1]
        
    # Convert tract polygon to the envelope polygon
    if earth_engine_clip_image_to_geometry == False:
        sub['geometry'] = sub['geometry'].envelope

    sub.to_file(temp_envelope_data_dir)
    tracts = geemap.shp_to_ee(temp_envelope_data_dir)
    
#     ###### for fixed rectangle size #######
#     d = calculate_centroid_meters(sub)
#     rectangle_geometries = sub['GEOID'].apply(lambda x: makeRectangle(d[x], xDistance, yDistance, 'EPSG:32632', x))
#     ee_rectangle_geometries = ee.List(list(rectangle_geometries))
#     rect_feat_collection = ee.FeatureCollection(ee_rectangle_geometries)
    
    # export and import to change projection
#     out_shp = "tester.shp"
#     geemap.ee_export_vector(rect_feat_collection, out_shp, verbose=True)
#     rect_feat_collection = geemap.shp_to_ee(out_shp)
    
#     geoids = tracts.aggregate_array('GEOID')
#     tracts=rect_feat_collection
    
    geoids = tracts.aggregate_array('GEOID')
    result = geoids.map(get_img)
    
    # download loop
    for i in range(0, sub.shape[0]):
        geoid = shapefile.iloc[interval+i]['GEOID']
        
        subdir = f"{save_dir}/{geoid}"
        filename = f"{subdir}/" + fn
        
#         if os.path.exists(filename):
#             continue
            
        init_dir(subdir, filename)
         
        img = ee.Image(result.get(i))
        
        img = img.select(['B2', 'B3', 'B4'])
        
        print(geoid)
#         for band in (['B2', 'B3', 'B4']:
#             img = img.select(band)
#             filename = f"{subdir}/" + fn + '/' + band + '.tif'
#             init_dir(subdir, filename)

    #         print(img)

        geemap.ee_export_image(
                img, filename=filename, file_per_band=False,  # region is for SKYSAT
            )
            

#        
    print(f'-----DONE {interval} - {interval + sub.shape[0]}')
print(f'Number of images with scale not at 30 m resolution: {count}')
    

/home/julia/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


09009350400
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009350400/sentinel2.tif
09009350500
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009350500/sentinel2.tif
09009352701
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009352701/sentinel2.tif
09009352702
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009352702/sentinel2.tif
09009352800
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009352800/sentinel2.tif
09009166002
Generating URL ...
Please wait ...
Data downloaded to

KeyboardInterrupt: 

# To download Sentinel

In [74]:
def maskClouds(image):
    # Get the cloud mask from the QA60 band
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Set the mask to include clear and shadow pixels
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0)) \
        .And(image.select('SCL').neq(3)) \
        .And(image.select('SCL').neq(8)) \
        .And(image.select('SCL').neq(9))
    # Apply the mask to the image
    return image.updateMask(mask)

def get_sentinel_img(geoid):
    feat = tracts.filter(ee.Filter.eq('GEOID', geoid)).first()
    roi = feat.geometry()
    
    collection = (
          ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")#('SKYSAT/GEN-A/PUBLIC/ORTHO/RGB')  #('LANDSAT/LC08/C02/T1_TOA')# # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1
          .filterBounds(roi)
#           .filter(ee.Filter.contains('.geo', roi))
          .filter(ee.Filter.calendarRange(6, 10, 'month'))
          .filterDate('2017-01-01', '2023-01-01')
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 3)) 
          .sort('system:time_start', False)
        )
    
    
    # Map the maskClouds function over the image collection
    maskedCollection = collection.map(maskClouds)

    # Create a composite of cloud-free pixels
    composite = maskedCollection.median()

    # composite = collection.limit(1).first()
    # # composite = high_quality.median()
    img = composite.select(['B2', 'B3', 'B4']).clip(roi)

#     img = top.first().clip(roi)#.reproject(crs='EPSG:4326')
    return img#, geoid
    

In [ ]:
save_dir = 'download_landsat_images'
count = 0
interval_size = 30
for interval in range(0, shapefile.shape[0], interval_size): 
    try:
        sub = shapefile.iloc[interval:interval+interval_size]
    except:
        sub = shapefile.iloc[interval:-1]
        
    # Convert tract polygon to the envelope polygon
    if earth_engine_clip_image_to_geometry == False:
        sub['geometry'] = sub['geometry'].envelope

    sub.to_file(temp_envelope_data_dir)
    tracts = geemap.shp_to_ee(temp_envelope_data_dir)
    
#     ###### for fixed rectangle size #######
#     d = calculate_centroid_meters(sub)
#     rectangle_geometries = sub['GEOID'].apply(lambda x: makeRectangle(d[x], xDistance, yDistance, 'EPSG:32632', x))
#     ee_rectangle_geometries = ee.List(list(rectangle_geometries))
#     rect_feat_collection = ee.FeatureCollection(ee_rectangle_geometries)
    
    # export and import to change projection
#     out_shp = "tester.shp"
#     geemap.ee_export_vector(rect_feat_collection, out_shp, verbose=True)
#     rect_feat_collection = geemap.shp_to_ee(out_shp)
    
#     geoids = tracts.aggregate_array('GEOID')
#     tracts=rect_feat_collection
    
    geoids = tracts.aggregate_array('GEOID')
    result = geoids.map(get_sentinel_img)
    
    # download loop
    for i in range(0, sub.shape[0]):
        geoid = shapefile.iloc[interval+i]['GEOID']
        
        subdir = f"{save_dir}/{geoid}"
        filename = f"{subdir}/" + fn
        
#         if os.path.exists(filename):
#             continue
            
        init_dir(subdir, filename)
        img = ee.Image(result.get(i))
        
        img = img.select(['B2', 'B3', 'B4'])
        
        feat = tracts.filter(ee.Filter.eq('GEOID', geoid)).first()
        roi = feat.geometry()
        
        geemap.ee_export_image(
                img, filename=filename, file_per_band=False, scale=10, region=roi # region is for SKYSAT
            )
#     if interval % 500 == 0:
    print(f'-----DONE {interval} - {interval + sub.shape[0]}')   

Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009350400/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009350500/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009352701/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009352702/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009352800/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/

Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009352100/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09005320200/sentinel2.tif
Generating URL ...
An error occurred while downloading.
Total request size (58707504 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09005342100/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09005310602/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09005250100/sentinel2.tif
Generating URL ...
Ple

Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009184200/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09005310803/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09005310804/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009140900/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009175800/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009344100

Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009175400/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009190301/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009343102/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009344200/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009352300/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009160200

Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009165500/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001230502/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001230501/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009184101/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009167101/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_im

...

Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001245300/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001022000/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001061200/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001070300/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001070500/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001070600

Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001220200/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001230400/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001245400/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001100200/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001090100/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_im

Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001030200/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001045102/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001010900/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001100100/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001205200/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001230100

Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001060300/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001090500/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001110201/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001030100/sentinel2.tif
-----DONE 330 - 360
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09001090200/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine

Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009150100/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009175600/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009175700/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009175900/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009180300/sentinel2.tif
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009180400

# For debugging

In [59]:
interval = 458
sub = shapefile.iloc[interval:interval+20]
sub['geometry'] = sub['geometry'].envelope

sub.to_file(temp_envelope_data_dir)
tracts = geemap.shp_to_ee(temp_envelope_data_dir)

geoids = tracts.aggregate_array('GEOID')
result = geoids.map(get_img)

In [55]:
42103950701

In [60]:
i=0
geoid = '42103950701' #shapefile.iloc[interval+i]['GEOID']

In [61]:
# geemap.ee_export_image(img, filename='test_sentinel.tif',  region=roi, scale=10)

In [72]:
feat = tracts.filter(ee.Filter.eq('GEOID', geoid)).first()
roi = feat.geometry()

collection = (
      ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")#('SKYSAT/GEN-A/PUBLIC/ORTHO/RGB')  #('LANDSAT/LC08/C02/T1_TOA')# # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1
      .filterBounds(roi)
#       .filter(ee.Filter.contains('.geo', roi))
      .filter(ee.Filter.calendarRange(6, 10, 'month'))
      .filterDate('2017-01-01', '2023-01-01')
#       .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 0.5)) 
      .sort('system:time_start', False)
      .sort('CLOUD_COVER')
    )


#  Print the number of images in the high-quality collection
print('Number of high-quality images:', collection.size().getInfo())

# Map the maskClouds function over the image collection
maskedCollection = collection.map(maskClouds)

# Create a composite of cloud-free pixels
composite = maskedCollection.median()

# composite = collection.limit(1).first()
# # composite = high_quality.median()
img = composite.select(['B2', 'B3', 'B4']).clip(roi)

# # top = high_quality.limit(1)
# # img = top.first().clip(roi)

# # img = collection.median().clip(roi)

# geemap.ee_export_image(img, filename='test_sentinel.tif',  region=roi, scale=10)

# # plot all
Map = geemap.Map()
# c = img.getInfo()['properties']['system:footprint']['coordinates'][0][0]
# Map.setCenter(c[0], c[1], 15)
vis_params = {'min': 0, 'max': 1500} #0, 0.3 for landsat
Map.addLayer(img, vis_params, 'Sentinel2')
Map.addLayerControl()
Map

Number of high-quality images: 8


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [63]:
img

In [17]:
img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [148]:
# Copy code
# Select the first image in the collection
# image = ee.Image(collection.first())

# # Extract RGB bands
# rgb = image.select(['B4', 'B3', 'B2'])

# Display the image on a map using geemap
Map = geemap.Map()
Map.addLayer(img, {'min':0, 'max':0.3}, 'RGB')
Map.addLayerControl()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [120]:
# Define the location of interest as a point
point = ee.Geometry.Point(-122.262, 37.8719)

# Define a start and end date
start_date = '2021-01-01'
end_date = '2021-01-31'

# Define the Landsat 8 collection
landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

# Filter the collection by location and date
image = ee.Image(landsat.filterBounds(point).filterDate(start_date, end_date).first())

# Define visualization parameters
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}

# Display the image on the map using geemap
Map = geemap.Map()
Map.centerObject(point, 10)
Map.addLayer(image, vis_params, 'Landsat 8')
Map.addLayer(point, {'color': 'red'}, 'Location')
Map.addLayerControl()
Map

Map(center=[37.8719, -122.262], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

# Run for rectangle census tract images

In [ ]:
def get_regions_to_retry():
    l = []
    counter=0
    for root, dirs, files in os.walk("download_landsat_images/"):
        if '.ipynb_checkpoints' in root:
            continue
        if fn not in files:
            l.append(root)
            continue
        else:
            counter+=1
        
            
    l.remove('download_landsat_images/')
    al = len(os.listdir("download_landsat_images/"))
    print(f'Num folders total: {al}')
    print(f'Num target files total: {counter}')
    print()
    l = [item[-11:] for item in l]
    return l
    
r = get_regions_to_retry()
print(len(r))
# for geoid in [r]:
#     shapefile.loc[shapefile.GEOID == geoid].to_file(temp_tract_data_dir)
#     tracts = geemap.shp_to_ee(temp_tract_data_dir)
    
#     # Convert tract polygon to the envelope polygon
#     envelope_shp = shapefile.loc[shapefile.GEOID == geoid]
#     envelope_shp['geometry'] = envelope_shp['geometry'].envelope
#     envelope_shp.to_file(temp_envelope_data_dir)
#     envelope_tracts = geemap.shp_to_ee(temp_envelope_data_dir)
    
#     img = get_img(geoid, envelope_tracts)
#     save_dir = 'download_landsat_images'
#     subdir = f"{save_dir}/{geoid}"
#     filename = f"{subdir}/" + fn

# #     img = ee.Image(result.get(i))
#     img = img.select(['B2', 'B3', 'B4', 'B5'])

#     try:
#         if img.projection().nominalScale().getInfo() != 30:
#             count +=1
#             continue
#     except:
#         print('--->Error getting projection. Passing...')

#     geemap.ee_export_image(
#             img, filename=filename, file_per_band=False)


#     print('--\n')

In [ ]:
df.shape[0]

In [ ]:
# # plot all
# Map = geemap.Map()
# Map.setCenter(-74.0060, 40.7128, 8)
# visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
# Map.addLayer(collection.first(), visParams, 'Landsat8')
# Map

# Test package



In [ ]:
cld

In [ ]:
img

In [ ]:
ee.Initialize()

aoi = ee.Geometry(Point([-48.4, -23]))

clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterDate('2019-11-25', '2019-11-28').filterBounds(aoi)

# img = s2.first()
cld = clouds.first()

Map = geemap.Map(center=(-23, -48.4), zoom=9)
Map.addLayer(cld, {'min': 0, 'max': 100, 'palette': ['purple', 'yellow']}, 'Cloud Probability', True, 1)
Map

## Scratchwork

In [ ]:
import tqdm


stats = pd.DataFrame(columns = [
    'geoid',
    'max',
 'mean',
 'min',
 'sample_sd',
 'sample_var',
 'sum',
 'sum_sq',
 'total_count',
 'total_sd',
 'total_var',
 'valid_count',
 'weight_sum',
 'weighted_sum'])

img_list = ee.List([])


def save_images(geoid):
    print(geoid_list)
    for geoid in geoid_list:
        print(geoid)
    save_dir = 'download_landsat_images/'
    subdir = f"{save_dir}{geoid}"


    if os.path.exists(subdir):
        if len(os.listdir(subdir)) >= 10:
#               print('pass')
          return True
        else:
#               print(f'Redoing: {subdir}')
            pass

    if not os.path.exists(subdir):
#             print('creating directory')
        createDir(subdir)

    
    geemap.ee_export_image(img, filename=f'{subdir}/least_CC.tif', region=roi, scale=90)

    # for image collection stats 
#         temp = pd.DataFrame(data=img.toArray().getInfo()).T
#         temp.columns=temp.keys().getInfo()
#         temp['geoid'] = geoid

#         stats = pd.concat([stats, temp], axis=0)
#         stats.to_csv('save_dir/_CC_stats.csv', index=False)


#         for img_idx in top.aggregate_array('system:index').getInfo():
#             img = ee.Image('LANDSAT/LC08/C02/T1_TOA/' + img_idx)
#             filename = f"{subdir}/{img_idx}.tif"

#             new_img = img.clip(roi)

#             geemap.ee_export_image(
#                 new_img, filename=filename, scale=90, region=roi, file_per_band=False
#             )

    return 

In [ ]:
# plot all
Map = geemap.Map()
Map.setCenter(-74.0060, 40.7128, 8)
visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
Map.addLayer(collection.first(), visParams, 'Landsat8')
# Map.addLayer(roi, {}, 'Census Tracts')
Map

In [ ]:
Map = geemap.Map()
Map.setCenter(-74.0060, 40.7128, 8)
visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
Map.addLayer(new_img, visParams, 'Landsat8')
# Map.addLayer(roi, {}, 'Census Tracts')
Map

In [ ]:
save_dir = 'download_landsat_images/'
def Count_files_in_subd():
    l = []
    
    for root, dirs, files in os.walk(save_dir):
        # print("{} in {}".format(len(files), root))
        l.append(len(files))
#         if len(files) == 0:
#           print(dirs)
    al = len(os.listdir(save_dir))
    print(f'Num folders total: {al}')
    print(f'Num folders with 0 files: {l.count(0)}')
    print(f'Num folders with >0 files: {al - l.count(0)}')
    print()
Count_files_in_subd()